In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flipkart-products/flipkart_com-ecommerce_sample.csv


## Importing Necessary Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading Data

In [3]:
df=pd.read_csv("/kaggle/input/flipkart-products/flipkart_com-ecommerce_sample.csv", na_values=["No rating available"])

In [4]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,NaN,NaN,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,NaN,NaN,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,NaN,NaN,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           1849 non-null   float64
 12  overall_rating           1849 non-null   float64
 13  brand                    14136 non-null  object 
 14  product_specifications

I have 3 choices of recommender system
1. Content Based - Recommends products based on Product Similarity
2. Collaborative Filtering - Recommends products based on User Preferences Similarities
3. Reinforcement Learning - Recommends product by going through purchase history of user by exploration and exploitation of different item recommendations and develops its learning.

Based on the given problem statement, we need to see user's preference / product and recommend similar products.

So the most suitable one here would be **Content Based Recommender system**

## Data Exploration

In [6]:
df.brand.nunique()

3499

In [7]:
df.product_category_tree.nunique()

6466

In [8]:
df.retail_price.describe()

count     19922.000000
mean       2979.206104
std        9009.639341
min          35.000000
25%         666.000000
50%        1040.000000
75%        1999.000000
max      571230.000000
Name: retail_price, dtype: float64

In [9]:
df.discounted_price.describe()

count     19922.000000
mean       1973.401767
std        7333.586040
min          35.000000
25%         350.000000
50%         550.000000
75%         999.000000
max      571230.000000
Name: discounted_price, dtype: float64

The discounts are pretty high

## Data Processing

In [10]:
df.product_category_tree[0]

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

In [11]:
df['product_category_tree']=df['product_category_tree'].map(lambda x:x.strip('[]'))
df['product_category_tree']=df['product_category_tree'].map(lambda x:x.strip('"'))
df['product_category_tree']=df['product_category_tree'].map(lambda x:x.split('>>'))

In [12]:
df.product_category_tree[0]

['Clothing ',
 " Women's Clothing ",
 ' Lingerie, Sleep & Swimwear ',
 ' Shorts ',
 ' Alisha Shorts ',
 " Alisha Solid Women's Cycling Shorts"]

In [13]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[Furniture , Living Room Furniture , Sofa Be...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,NaN,NaN,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[Footwear , Women's Footwear , Ballerinas , ...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,NaN,NaN,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[Pet Supplies , Grooming , Skin & Coat Care ...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,NaN,NaN,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


Category Heirarchy Tree has been converted to a list

In [14]:
# removing unwanted features
todelete=['crawl_timestamp','product_url','image',"is_FK_Advantage_product","product_rating","overall_rating","product_specifications"]
df=df.drop(todelete,axis=1)

In [15]:
df.head()

,uniq_id,product_name,product_category_tree,pid,retail_price,discounted_price,description,brand
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2FF9KEDEFGF,999.0,379.0,Key Features of Alisha Solid Women's Cycling S...,Alisha
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[Furniture , Living Room Furniture , Sofa Be...",SBEEH3QGU7MFYJFY,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[Footwear , Women's Footwear , Ballerinas , ...",SHOEH4GRSUBJGZXE,999.0,499.0,Key Features of AW Bellies Sandals Wedges Heel...,AW
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2F6HUZMQ6SJ,699.0,267.0,Key Features of Alisha Solid Women's Cycling S...,Alisha
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[Pet Supplies , Grooming , Skin & Coat Care ...",PSOEH3ZYDMSYARJ5,220.0,210.0,Specifications of Sicons All Purpose Arnica Do...,Sicons


In [16]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
exclude = set(string.punctuation)
import string

In [17]:
df.shape

(20000, 8)

In [18]:
dfcopy=df.copy()
# removing duplicate product rows
dfcopy.drop_duplicates(subset ="product_name", 
                     keep = "first", inplace = True)
dfcopy.shape

(12676, 8)

8000 rows got removed

In [19]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
from nltk.corpus import wordnet

In [21]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [22]:
def filter_keywords(doc):
    doc=doc.lower()
    stop_free = " ".join([i for i in doc.split() if i not in stop_words])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    word_tokens = word_tokenize(punc_free)
    filtered_sentence = [(lem.lemmatize(w, "v")) for w in word_tokens]
    return filtered_sentence

In [23]:
dfcopy['product'] = dfcopy['product_name'].apply(filter_keywords)
dfcopy['description'] = dfcopy['description'].astype("str").apply(filter_keywords)
dfcopy['brand'] = dfcopy['brand'].astype("str").apply(filter_keywords)

In [24]:
# combining all the product metadata under one column
dfcopy["all_meta"]=dfcopy['product']+dfcopy['brand']+ df['product_category_tree']+dfcopy['description']
dfcopy["all_meta"] = dfcopy["all_meta"].apply(lambda x: ' '.join(x))

In [25]:
dfcopy["all_meta"].head()

0    alisha solid womens cycle short alisha Clothin...
1    fabhomedecor fabric double sofa bed fabhomedec...
2    aw belly aw Footwear   Women's Footwear   Ball...
4    sicons purpose arnica dog shampoo sicons Pet S...
5    eternal gandhi super series crystal paper weig...
Name: all_meta, dtype: object

In [26]:
dfcopy.head()

,uniq_id,product_name,product_category_tree,pid,retail_price,discounted_price,description,brand,product,all_meta
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[Clothing , Women's Clothing , Lingerie, Sle...",SRTEH2FF9KEDEFGF,999.0,379.0,"[key, feature, alisha, solid, womens, cycle, s...",[alisha],"[alisha, solid, womens, cycle, short]",alisha solid womens cycle short alisha Clothin...
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[Furniture , Living Room Furniture , Sofa Be...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[fabhomedecor, fabric, double, sofa, bed, fini...",[fabhomedecor],"[fabhomedecor, fabric, double, sofa, bed]",fabhomedecor fabric double sofa bed fabhomedec...
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[Footwear , Women's Footwear , Ballerinas , ...",SHOEH4GRSUBJGZXE,999.0,499.0,"[key, feature, aw, belly, sandals, wedge, heel...",[aw],"[aw, belly]",aw belly aw Footwear Women's Footwear Ball...
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[Pet Supplies , Grooming , Skin & Coat Care ...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[specifications, sicons, purpose, arnica, dog,...",[sicons],"[sicons, purpose, arnica, dog, shampoo]",sicons purpose arnica dog shampoo sicons Pet S...
5,c2a17313954882c1dba461863e98adf2,Eternal Gandhi Super Series Crystal Paper Weig...,[Eternal Gandhi Super Series Crystal Paper Wei...,PWTEB7H2E4KCYUE3,430.0,430.0,"[key, feature, eternal, gandhi, super, series,...","[eternal, gandhi]","[eternal, gandhi, super, series, crystal, pape...",eternal gandhi super series crystal paper weig...


In [27]:
dfcopy.to_csv("/kaggle/working/preprocessed.csv", index=False)

In [28]:
dfcopy.columns

Index(['uniq_id', 'product_name', 'product_category_tree', 'pid',
       'retail_price', 'discounted_price', 'description', 'brand', 'product',
       'all_meta'],
      dtype='object')

 Data has been preprocessed and saved as a new csv

## Calculating Product Similarities

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(dfcopy['all_meta'])

In [30]:
# using cosine similarities for calculating similarities between 2 product metadata vectors
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [31]:
cosine_sim

array([[1.00000000e+00, 8.68316315e-03, 1.51436535e-02, ...,
        5.45842099e-03, 7.31208048e-03, 1.31609786e-03],
       [8.68316315e-03, 1.00000000e+00, 5.19867277e-02, ...,
        1.03477740e-03, 1.38618396e-03, 3.22026556e-02],
       [1.51436535e-02, 5.19867277e-02, 1.00000000e+00, ...,
        4.55013031e-02, 7.25525053e-02, 1.62818750e-02],
       ...,
       [5.45842099e-03, 1.03477740e-03, 4.55013031e-02, ...,
        1.00000000e+00, 1.27188991e-01, 1.60111453e-04],
       [7.31208048e-03, 1.38618396e-03, 7.25525053e-02, ...,
        1.27188991e-01, 1.00000000e+00, 2.14484707e-04],
       [1.31609786e-03, 3.22026556e-02, 1.62818750e-02, ...,
        1.60111453e-04, 2.14484707e-04, 1.00000000e+00]])

## Recommendations

### Without Price Filter

In [32]:
dfcopy = dfcopy.reset_index()
titles = dfcopy['product_name']
indices = pd.Series(dfcopy.index, index=dfcopy['product_name'])

In [33]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    product_indices = [i[0] for i in sim_scores]
    return titles.iloc[product_indices]

In [34]:
get_recommendations("FabHomeDecor Fabric Double Sofa Bed").head(5)

12219    Comfort Couch Engineered Wood 3 Seater Sofa
12199        @home Annulus Solid Wood Dressing Table
11866       Ethnic Handicrafts Solid Wood Single Bed
11857        Ethnic Handicrafts Solid Wood Queen Bed
5191                    HomeEdge Solid Wood King Bed
Name: product_name, dtype: object

### With Price Filter

In [35]:
def get_recommendations2(title, min_price, max_price):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    product_indices = [i[0] for i in sim_scores]

    # Filter products by price range
    filtered_indices = [i for i in product_indices if min_price <= dfcopy['discounted_price'].iloc[i] <= max_price]

    # Take the top 5 items from the filtered list
    recommended_indices = filtered_indices[:5]

    return titles.iloc[recommended_indices]

In [36]:
get_recommendations2("FabHomeDecor Fabric Double Sofa Bed",1,1000).head(5)

12346                   Lovely Plastic Desk Chair
5147            Anmol Cotton Collapsible Wardrobe
7469         Varmora VCST0 Living & Bedroom Stool
6387        Palakz Printed Foldable Storage Stool
11940    SRK GROUPS Aluminium, Wooden Home Temple
Name: product_name, dtype: object

### With Discount Percent Filter

In [37]:
def get_recommendations3(title, min_price, max_price, min_discount_percent):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    product_indices = [i[0] for i in sim_scores]

    # Filter products by price range and minimum discount percent
    filtered_indices = [
        i for i in product_indices
        if min_price <= dfcopy['discounted_price'].iloc[i] <= max_price
        and (dfcopy['retail_price'].iloc[i] - dfcopy['discounted_price'].iloc[i]) / dfcopy['retail_price'].iloc[i] * 100 >= min_discount_percent
    ]

    # Take the top 5 items from the filtered list
    recommended_indices = filtered_indices[:5]

    return titles.iloc[recommended_indices]

In [38]:
get_recommendations3("FabHomeDecor Fabric Double Sofa Bed",1,1000, 10).head(5)

12346                   Lovely Plastic Desk Chair
5147            Anmol Cotton Collapsible Wardrobe
7469         Varmora VCST0 Living & Bedroom Stool
6387        Palakz Printed Foldable Storage Stool
11940    SRK GROUPS Aluminium, Wooden Home Temple
Name: product_name, dtype: object